# Sudoku Solver
- Used OpenCV tools to preprocess the image and extract the digits in the sudoku grid image
- Used SAT solver to solve the sudoku grid

In [ ]:
## standard imports
import tensorflow as tf
import cv2 as cv
import numpy as np
import operator
from z3 import *

## A CNN model to get digits from image

In [ ]:
def returnmodel():
    model = tf.keras.Sequential([
                                 
        tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
        
        tf.keras.layers.Conv2D(64, (2, 2), activation="relu", padding="same"),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(128, (2, 2), activation="relu", padding="same"),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(256, (2, 2), activation="relu", padding="same"),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

#### Loading the dataset to train the CNN

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()  ## loading the data
x_train = x_train / 255.0        ## normalising the dataset
x_test = x_test / 255.0         ## normalising the dataset

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
x_train = np.expand_dims(x_train, axis = 3)
x_test = np.expand_dims(x_test, axis = 3)
x_train.shape, x_test.shape

In [ ]:
cv.imshow('image', x_train[5])
cv.waitKey(0)

#### Training the model

In [ ]:
model = returnmodel()
EPOCHS = 15
BATCHSIZE = 32
model.fit(x_train, y_train, epochs = EPOCHS, batch_size = BATCHSIZE, validation_split= 0.1)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
prediction = model.predict(np.expand_dims(x_test[11], axis = 0))
np.argmax(prediction)

In [ ]:
cv.imshow('image', x_test[11])
cv.waitKey(0)

## Preprocessing

In [ ]:
def preprocess(img):
  
  img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)                ## converting the image into grayscale
  blur = cv.GaussianBlur(img_gray, (9, 9), 0)                   ## blurring the image
  thresh = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2) # thresholding it
  inverted = cv.bitwise_not(thresh, 0)                          ## inverting the image
  kernel = cv.getStructuringElement(cv.MORPH_RECT, (2, 2))      ## to get a kernel
  morph = cv.morphologyEx(inverted, cv.MORPH_OPEN, kernel)      ## morphing it to remove random noise
  result = cv.dilate(morph, kernel, iterations=1)               ## dilating the image to increase the border size
  return result

## Extracting the digits from image

### Utility functions

In [ ]:
## function to identify the corners of the grid

def extremecorners(polygon, limit_fn, compare_fn):
    # limit_fn is the min or max function
    # compare_fn is the np.add or np.subtract function
    # if we are trying to find bottom left corner, we know that it will have the smallest (x - y) value
    section, _ = limit_fn(enumerate([compare_fn(pt[0][0], pt[0][1]) for pt in polygon]),
                          key=operator.itemgetter(1))

    return polygon[section][0][0], polygon[section][0][1]


In [ ]:
## function to label the sudoku grid

def extrememarkers(pts, original):
    cv.circle(original, pts, 5, (0, 255, 0), cv.FILLED)

In [ ]:
## function to get grid lines

def gridlines(img, location, length=10):
    clone = img.copy()
    roworcol = clone.shape[location]   # location is 1 for horizontal and 0 for vertical 
    size = roworcol // length         # to find distance between lines so as to get good grid size

    if location == 0:                   # to get structuring element depending upon whether it is horizontal or vertical
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (1, size))
    else:
        kernel = cv.getStructuringElement(cv.MORPH_RECT, (size, 1))

    clone = cv.erode(clone, kernel)
    clone = cv.dilate(clone, kernel)

    return clone

In [ ]:
## function to draw lines, i.e, to draw grid lines

def drawlines(img, lines):
    
    clone = img.copy()
    lines = np.squeeze(lines)

    for rho, theta in lines:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * a)
        cv.line(clone, (x1, y1), (x2, y2), (255, 255, 255), 4)
    return clone

In [ ]:
def cleanhelper(img):
    if np.isclose(img, 0).sum() / (img.shape[0] * img.shape[1]) >= 0.95:
        return np.zeros_like(img), False

    # if there is very little white in the region around the center, this means we got an edge accidently
    height, width = img.shape
    mid = width // 2
    if np.isclose(img[:, int(mid - width * 0.4):int(mid + width * 0.4)], 0).sum() / (2 * width * 0.4 * height) >= 0.90:
        return np.zeros_like(img), False

    # center image
    contours, _ = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv.contourArea, reverse=True)
    x, y, w, h = cv.boundingRect(contours[0])

    startx = (width - w) // 2
    starty = (height - h) // 2
    newimg = np.zeros_like(img)
    newimg[starty:starty + h, startx:startx + w] = img[y:y + h, x:x + w]

    return newimg, True

In [ ]:
## function to get grid mask

def gridmask(vertical, horizontal):
    grid = cv.add(horizontal, vertical)    # combine the vertical and horizontal lines to make a grid
    
    # threshold and dilate the grid to cover more area
    grid = cv.adaptiveThreshold(grid, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 235, 2)
    grid = cv.dilate(grid, cv.getStructuringElement(cv.MORPH_RECT, (3, 3)), iterations=2)

    pts = cv.HoughLines(grid, .3, np.pi / 90, 200)     # to find where lines are located

    lines = drawlines(grid, pts)
    mask = cv.bitwise_not(lines)   # extracting the lines so that only numbers are left
    return mask

In [ ]:
# function to get grid lines

def getgridlines(img, length=10):
    horizontal = gridlines(img, 1, length)
    vertical = gridlines(img, 0, length)
    return vertical, horizontal

In [ ]:
# function to find contours

def find_contours(img, original):

    contours, _ = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=cv.contourArea, reverse=True)      # sorting the contours to get the largest polygon
    
    polygon = None

    # make sure this is the one we are looking for
    for cnt in contours:
        area = cv.contourArea(cnt)
        perimeter = cv.arcLength(cnt, closed=True)
        approx = cv.approxPolyDP(cnt, 0.01 * perimeter, closed=True)
        num_corners = len(approx)

        if num_corners == 4 and area > 1000:
            polygon = cnt
            break

    if polygon is not None:
        # find its extreme corners
        topleft = extremecorners(polygon, min, np.add)  # has smallest (x + y) value
        topright = extremecorners(polygon, max, np.subtract)  # has largest (x - y) value
        botleft = extremecorners(polygon, min, np.subtract)  # has smallest (x - y) value
        botright = extremecorners(polygon, max, np.add)  # has largest (x + y) value

        # if its not a square, we don't want it
        if botright[1] - topright[1] == 0:
            return []
        if not (0.95 < ((topright[0] - topleft[0]) / (botright[1] - topright[1])) < 1.05):
            return []

        cv.drawContours(original, [polygon], 0, (0, 0, 255), 3)

        # drawing corresponding circles just to make sure we had correctly identified it
        [extrememarkers(x, original) for x in [topleft, topright, botright, botleft]]

        return [topleft, topright, botright, botleft]

    return []

In [ ]:
# function to warp the image

def warpimage(corners, original):
    
    corners = np.array(corners, dtype='float32')
    topleft, topright, botright, botleft = corners

    # find the best side width, since we will be warping into a square, height = length
    width = int(max([
        np.linalg.norm(topright - botright),
        np.linalg.norm(topleft - botleft),
        np.linalg.norm(botright - botleft),
        np.linalg.norm(topleft - topright)
    ]))

    # creating an array with shows top_left, top_right, bot_left, bot_right
    mapping = np.array([[0, 0], [width - 1, 0], [width - 1, width - 1], [0, width - 1]], dtype='float32')

    matrix = cv.getPerspectiveTransform(corners, mapping)

    return cv.warpPerspective(original, matrix, (width, width)), matrix

In [ ]:
# function to split the warped image into squares

def splittosquares(warpedimg):
    squares = []

    width = warpedimg.shape[0] // 9

    # finding each square by assuming they are of the same side
    for j in range(9):
        for i in range(9):
            p1 = (i * width, j * width)  # Top left corner of a bounding box
            p2 = ((i + 1) * width, (j + 1) * width)  # Bottom right corner of bounding box
            squares.append(warpedimg[p1[1]:p2[1], p1[0]:p2[0]])

    return squares


In [ ]:
## function to get images of digits if a digit is present
## the values are appended into a list

def cleansquares(squares):
    cleanedsquares = []

    for square in squares:
        newimg, isnumber = cleanhelper(square)
        if isnumber:      # if there is a non-zero number, appending the image to the list
            cleanedsquares.append(newimg)
        else:
            cleanedsquares.append(0)

    return cleanedsquares

In [ ]:
## function to recognise the digits

def recognizedigits(squares, model):
    s = ""    # a string to store the recognised digits
    formattedimages = []
    zeropositions = set()

    blankimage = np.zeros_like(cv.resize(squares[0], (28, 28)))
    blankimage = np.expand_dims(blankimage, axis = 2)

    # adding the list of splitted square images so that we can predict later
    for i in range(len(squares)):
        if type(squares[i]) == int:
            zeropositions.add(i)
            formattedimages.append(blankimage)
        else:
            print(squares[i].shape)
            img = cv.resize(squares[i], (28, 28))
#             cv.imshow('image', img)
#             cv.waitKey(0)
#             print(img.shape)
            img = np.expand_dims(img, axis = 2)
            cv.imshow('image', img)
            cv.waitKey(0)
            print(img.shape)
            prediction = model.predict(np.expand_dims(img, axis = 0))
            print(np.argmax(prediction))
            formattedimages.append(img)

    formattedimages = np.array(formattedimages)
    print(formattedimages.shape)
    all_preds = list(map(np.argmax, model(tf.convert_to_tensor(formattedimages))))
    print(all_preds)
    for i in range(len(all_preds)):
        if i in zeropositions:
            s += "0"
        else:
            s += str(all_preds[i] + 1)

    return s

In [ ]:
# function to draw digits on the image

def drawdigitsonwarped(warpedimg, solvedpuzzle, squaresprocessed):
    width = warpedimg.shape[0] // 9

    imgwithtext = np.zeros_like(warpedimg)

    # find each square assuming they are of the same side
    index = 0
    for j in range(9):
        for i in range(9):
            if type(squaresprocessed[index]) == int:
                p1 = (i * width, j * width)  # Top left corner of a bounding box
                p2 = ((i + 1) * width, (j + 1) * width)  # Bottom right corner of bounding box

                center = ((p1[0] + p2[0]) // 2, (p1[1] + p2[1]) // 2)
                text_size, _ = cv.getTextSize(str(solvedpuzzle[index]), cv.FONT_HERSHEY_SIMPLEX, 0.75, 4)
                text_origin = (center[0] - text_size[0] // 2, center[1] + text_size[1] // 2)

                cv.putText(warpedimg, str(solvedpuzzle[index]),
                            text_origin, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            index += 1

    return imgwithtext

## Sudoku SAT Solver 

In [ ]:
def solvepuzzle(instance):
    # 9x9 matrix of integer variables
    X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ]
          for i in range(9) ]

    # each cell contains a value in {1, ..., 9}
    cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
                 for i in range(9) for j in range(9) ]

    # each row contains a digit at most once
    rows_c   = [ Distinct(X[i]) for i in range(9) ]

    # each column contains a digit at most once
    cols_c   = [ Distinct([ X[i][j] for i in range(9) ])
                 for j in range(9) ]

    # each 3x3 square contains a digit at most once
    sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                            for i in range(3) for j in range(3) ])
                 for i0 in range(3) for j0 in range(3) ]
    
    sudoku_c = cells_c + rows_c + cols_c + sq_c        ## overall conditions for sudoku 
    instance_c = [ If(instance[i][j] == 0,
                      True,
                      X[i][j] == instance[i][j])
                   for i in range(9) for j in range(9) ]

    s = Solver()
    s.add(sudoku_c + instance_c)
    if s.check() == sat:
        m = s.model()
        r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
              for i in range(9) ]
        return r, 1
    else:
        return 0, 0

In [ ]:
def getinstance(prediction):
    l = []
    for i in prediction:
        l += [int(i)]
    arr = np.array(l)
    arr1 = arr.reshape(9,9)
    return arr1

In [ ]:
sudokuimg = cv.imread('./images/sudoku1.png')
imgcorners = sudokuimg.copy()
cv.imshow('image', sudokuimg)
cv.waitKey(0)

In [ ]:
processedimg = preprocess(sudokuimg)
corners = find_contours(processedimg, imgcorners)


In [ ]:
cv.imshow('image', processedimg)
cv.waitKey(0)

In [ ]:
if corners:
    warped, matrix = warpimage(corners, sudokuimg)
    warpedprocessed = preprocess(warped)
    
    verticallines, horizontallines = getgridlines(warpedprocessed)
    mask = gridmask(verticallines, horizontallines)
    numbers = cv.bitwise_and(warpedprocessed, mask)

    squares = splittosquares(numbers)

    squaresprocessed = cleansquares(squares)
    
    mymodel = returnmodel()
    squaresprediction = recognizedigits(squaresprocessed, mymodel)

    solvedpuzzle = sudoku.solve_wrapper(squares_guesses)
    instance = getinstance(squaresprediction)
    solvedpuzzle, checker = solvepuzzle(instance)
    if checker != 0:
        drawdigitsonwarped(warped, solvedpuzzle, squaresprocessed)
    else:
        print "Unsolvable Sudoku Puzzle"